# **레스토랑 리뷰 감성 분류하기**

<img src = "https://miro.medium.com/v2/resize:fit:1400/format:webp/1*2BvNKzXlJYGLofXC7a-9_A.png" height = 300 width = 600>

2015년에 옐프(Yelp)가 주최한 레스토랑 등급 예측 대회에서 사용되었던 레스토랑 리뷰 데이터를 긍정, 부정으로 분류합니다.

저희는 **전처리된 데이터셋**을 사용하겠습니다.

## 1) 임포트

In [ ]:
from argparse import Namespace
from collections import Counter
import json
import os
import re
import string

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import tqdm

### 시드, 하이퍼파라미터 설정

`set_seed_everywhere()`로 random seed를 동일하게 설정하고 `handle_dirs()`로 dirpath가 존재하는 디렉터리인지 확인하고, 존재하지 않는다면 직접 만듭니다.

In [ ]:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

모델 생성/훈련 과정에서 사용할 변수를 설정합니다.
- 학습 데이터셋에서 25번 이상 출현한 단어만 학습하고 나머지는 UNK으로 처리하도록 frequency_cutoff를 설정해주세요.
- batch size는 128, learning rate는 0.001, epoch의 수는 10으로 지정합니다.
- GPU를 사용하도록 cuda를 지정해주세요(T/F).

In [ ]:
args = Namespace(
# =======================================
# Q1
# =======================================
    frequency_cutoff=25,

    # 모델 상태를 저장할 파일명
    model_state_file='model.pth',

    # 리뷰 데이터셋 csv의 위치
    review_csv='data/yelp/reviews_with_splits_lite.csv',

    # vectorizer_file과 model_state_file을 저장할 위치
    save_dir='model_storage/ch3/yelp/',
    vectorizer_file='vectorizer.json',
    
    # 훈련 하이퍼파라미터
    batch_size=128,
    early_stopping_criteria=5,
    learning_rate=0.001,
    num_epochs=100,
    seed=1337,
# =======================================
# Q2
# =======================================
    # 실행 옵션
    catch_keyboard_interrupt=True,
    cuda=True,
    expand_filepaths_to_save_dir=True,
    reload_from_files=False,
)

GPU를 사용하는지 확인해봅시다.

In [ ]:
# 파일 경로
if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir,
                                        args.vectorizer_file)

    args.model_state_file = os.path.join(args.save_dir,
                                         args.model_state_file)
    
    print("파일 경로: ")
    print("\t{}".format(args.vectorizer_file))
    print("\t{}".format(args.model_state_file))
    
# CUDA 체크
if not torch.cuda.is_available():
    args.cuda = False

print("CUDA 사용여부: {}".format(args.cuda))
args.device = torch.device("cuda" if args.cuda else "cpu")

# 재현성을 위해 시드 설정
set_seed_everywhere(args.seed, args.cuda)

# 디렉토리 처리
handle_dirs(args.save_dir)

파일 경로: 
	model_storage/ch3/yelp/vectorizer.json
	model_storage/ch3/yelp/model.pth
CUDA 사용여부: True


### 데이터셋 불러오기
데이터셋을 github에서 불러옵니다.

In [ ]:
# 코랩에서 실행하는 경우 아래 코드를 실행하여 전처리된 라이트 버전의 데이터를 다운로드하세요.
!mkdir data
!wget https://git.io/JtRSq -O data/download.py
!wget https://git.io/JtRSO -O data/get-all-data.sh
!chmod 755 data/get-all-data.sh
%cd data
!./get-all-data.sh
%cd ..

--2023-04-02 02:12:21--  https://git.io/JtRSq
Resolving git.io (git.io)... 140.82.113.21
Connecting to git.io (git.io)|140.82.113.21|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rickiepark/nlp-with-pytorch/main/chapter_3/data/download.py [following]
--2023-04-02 02:12:21--  https://raw.githubusercontent.com/rickiepark/nlp-with-pytorch/main/chapter_3/data/download.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1572 (1.5K) [text/plain]
Saving to: ‘data/download.py’

data/download.py    100%[===================>]   1.54K  --.-KB/s    in 0s      

2023-04-02 02:12:21 (16.9 MB/s) - ‘data/download.py’ saved [1572/1572]

--2023-04-02 02:12:22--  https://git.io/JtRSO
Resolving git.io (git.io)...

데이터를 간단하게 살펴봅시다. 이미 전처리되어 train, val, test로 구분되어 있습니다.

In [ ]:
review_df = pd.read_csv(args.review_csv)
review_df.head()

,rating,review,split
0,negative,terrible place to work for i just heard a stor...,train
1,negative,"hours , minutes total time for an extremely s...",train
2,negative,my less than stellar review is for service . w...,train
3,negative,i m granting one star because there s no way t...,train
4,negative,the food here is mediocre at best . i went aft...,train


In [ ]:
review_df.tail()

,rating,review,split
55995,positive,"great food . wonderful , friendly service . i ...",test
55996,positive,charlotte should be the new standard for moder...,test
55997,positive,get the encore sandwich ! ! make sure to get i...,test
55998,positive,i m a pretty big ice cream gelato fan . pretty...,test
55999,positive,where else can you find all the parts and piec...,test


In [ ]:
review_df.split.unique()

array(['train', 'val', 'test'], dtype=object)

## 2) 데이터 벡터화 클래스
이제 클래스를 정의해주겠습니다.

파이토치 클래스 이해를 돕기 위한 설명입니다!
- `Dataset` 클래스: 반복자(iterator) 클래스로 `__init__`를 통해 데이터를 처리해 가져오고 `__getitem__`으로 데이터를 하나씩 판독합니다. 파이토치에서 새로운 데이터셋을 사용할 때 먼저 `Dataset` 클래스를 상속해서 `__getitem()`와 ` __len__()` 메서드를 필수로 구현해야 합니다.

- `ReviewVectorizer` 클래스: 어휘 사전을 생성, 관리하는 역할로 리뷰 텍스트를 숫자로 매핑합니다.

- `DataLoader` 클래스: 데이터셋으로 미니배치를 만듭니다.
- 모든 클래스는 서로 의존합니다.



### `Dataset` 클래스
1. 저희는 이름을 살짝 바꾼 `ReviewDataset` 클래스를 사용하겠습니다.
2. 전처리된 데이터셋 `review_df (Dataset)`의 각 샘플은 위에서 확인한 것처럼 훈련, 검증, 테스트 중 어느 세트인지 표시되어 있습니다.
3. 리뷰를 공백을 기준으로 분리하여 토큰 리스트를 얻고자 합니다.
4. `ReviewVectorizer`와 `DataLoader`는 뒷 부분에서 정의할 클래스입니다. (클래스는 서로 의존함)

빈칸을 채워주세요. (튜토리얼: https://tutorials.pytorch.kr/beginner/basics/data_tutorial.html)

In [ ]:
class ReviewDataset(Dataset):
# =======================================
# Q3, 4, 5, 6
# =======================================

    def __init__(self, review_df, vectorizer):
        """ train_df / train_size / val_df / val_size / test_df / test_size 변수를 정의합니다
        매개변수:
            review_df (pandas.DataFrame): 데이터셋
            vectorizer (ReviewVectorizer): ReviewVectorizer 객체  
        """
        self.review_df = review_df
        self._vectorizer = vectorizer

        self.train_df = self.review_df[self.review_df.split=='train']
        self.train_size = len(self.train_df)

        self.val_df = self.review_df[self.review_df.split=='val']
        self.validation_size = len(self.val_df)

        self.test_df = self.review_df[self.review_df.split=='test']
        self.test_size = len(self.test_df)

        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size),
                             'test': (self.test_df, self.test_size)}

        self.set_split('train')

    @classmethod
    def load_dataset_and_make_vectorizer(cls, review_csv):
        """ 데이터셋을 로드하고 새로운 ReviewVectorizer 객체를 만듭니다
        
        매개변수:
            review_csv (str): 데이터셋의 위치
        반환값:
            ReviewDataset의 인스턴스
        """
        review_df = pd.read_csv(review_csv)
        train_review_df = review_df[review_df.split=='train']
        return cls(review_df, ReviewVectorizer.from_dataframe(train_review_df))
    
    @classmethod
    def load_dataset_and_load_vectorizer(cls, review_csv, vectorizer_filepath):
        """ 데이터셋을 로드하고 새로운 ReviewVectorizer 객체를 만듭니다.
        캐시된 ReviewVectorizer 객체를 재사용할 때 사용합니다.
        
        매개변수:
            review_csv (str): 데이터셋의 위치
            vectorizer_filepath (str): ReviewVectorizer 객체의 저장 위치
        반환값:
            ReviewDataset의 인스턴스
        """
        review_df = pd.read_csv(review_csv)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(review_df, vectorizer)

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        """ 파일에서 ReviewVectorizer 객체를 로드하는 정적 메서드
        
        매개변수:
            vectorizer_filepath (str): 직렬화된 ReviewVectorizer 객체의 위치
        반환값:
            ReviewVectorizer의 인스턴스
        """
        with open(vectorizer_filepath) as fp:
            return ReviewVectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_filepath):
        """ ReviewVectorizer 객체를 json 형태로 디스크에 저장합니다
        
        매개변수:
            vectorizer_filepath (str): ReviewVectorizer 객체의 저장 위치
        """
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self):
        """ 벡터 변환 객체를 반환합니다 """
        return self._vectorizer

    def set_split(self, split="train"):
        """ 데이터프레임에 있는 열을 사용해 분할 세트를 선택합니다 
        
        매개변수:
            split (str): "train", "val", "test" 중 하나
        """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

# =======================================
# Q7 데이터셋의 샘플 개수를 반환하는 이 함수는 무엇일까요?
# =======================================

    def __len__(self):
        return self._target_size

# =======================================
# Q8 주어진 인덱스 idx 에 해당하는 샘플을 데이터셋에서 불러오고 반환하는 이 함수는 무엇일까요?
# =======================================

    def __getitem__(self, index):
        """ 파이토치 데이터셋의 주요 진입 메서드
        
        매개변수:
            index (int): 데이터 포인트의 인덱스
        반환값:
            데이터 포인트의 특성(x_data)과 레이블(y_target)로 이루어진 딕셔너리
            {"x_data" : review_vector, "y_target" : rating_index} 형태
        """
        row = self._target_df.iloc[index]

        review_vector = \
            self._vectorizer.vectorize(row.review)

        rating_index = \
            self._vectorizer.rating_vocab.lookup_token(row.rating)

        return {'x_data': review_vector,
                'y_target': rating_index}

    def get_num_batches(self, batch_size):
        """ 배치 크기가 주어지면 데이터셋으로 만들 수 있는 배치 개수를 반환합니다
        
        매개변수:
            batch_size (int)
        반환값:
            배치 개수
        """
        return len(self) // batch_size

이제
1. 각 토큰(단어)을 정수에 매핑하고 (`Vocabulary` 클래스)
2. 이를 벡터 형태로 변환한 뒤 (`ReviewVectorizer` 클래스)
3. 모델에서 사용하기 위한 미니배치로 묶어주겠습니다. (`DataLoader` 클래스)

### `Vocabulary` 클래스

토큰과 정수를 1:1 매핑하고 이를 관리합니다. 

사용자가 딕셔너리에 새로운 토큰을 추가하면 자동으로 인덱스를 증가시키고, 훈련 데이터셋에 없는 토큰은 `UNK(Unknown)`으로 처리합니다.

In [ ]:
class Vocabulary(object):
    """ 매핑을 위해 텍스트를 처리하고 어휘 사전을 만드는 클래스 """

    def __init__(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"):
        """
        매개변수:
            token_to_idx (dict): 기존 토큰-인덱스 매핑 딕셔너리
            add_unk (bool): UNK 토큰을 추가할지 지정하는 플래그
            unk_token (str): Vocabulary에 추가할 UNK 토큰
        """

        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx

        self._idx_to_token = {idx: token 
                              for token, idx in self._token_to_idx.items()}
        
        self._add_unk = add_unk
        self._unk_token = unk_token
        
        self.unk_index = -1
        if add_unk:
            self.unk_index = self.add_token(unk_token) 
        
        
    def to_serializable(self):
        """ 직렬화할 수 있는 딕셔너리를 반환합니다 """
        return {'token_to_idx': self._token_to_idx, 
                'add_unk': self._add_unk, 
                'unk_token': self._unk_token}

    @classmethod
    def from_serializable(cls, contents):
        """ 직렬화된 딕셔너리에서 Vocabulary 객체를 만듭니다 """
        return cls(**contents)

# =======================================

    def add_token(self, token):
        """ 토큰을 기반으로 매핑 딕셔너리를 업데이트합니다 # 새로운 토큰을 추가하기 위한 함수

        매개변수:
            token (str): Vocabulary에 추가할 토큰
        반환값:
            index (int): 토큰에 상응하는 정수
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index

    def add_many(self, tokens):
        """ 토큰 리스트를 Vocabulary에 추가합니다.
        
        매개변수:
            tokens (list): 문자열 토큰 리스트
        반환값:
            indices (list): 토큰 리스트에 상응되는 인덱스 리스트
        """
        return [self.add_token(token) for token in tokens]

    def lookup_token(self, token):
        """ 토큰에 대응하는 인덱스를 추출합니다.
        토큰이 없으면 UNK 인덱스를 반환합니다.
        
        매개변수:
            token (str): 찾을 토큰 
        반환값:
            index (int): 토큰에 해당하는 인덱스
        노트:
            UNK 토큰을 사용하려면 (Vocabulary에 추가하기 위해)
            `unk_index`가 0보다 커야 합니다.
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

    def lookup_index(self, index):
        """ 인덱스에 대응하는 토큰을 찾아 반환합니다.
        
        매개변수: 
            index (int): 찾을 인덱스
        반환값:
            token (str): 인텍스에 해당하는 토큰
        에러:
            KeyError: 인덱스가 Vocabulary에 없을 때 발생합니다.
        """
        if index not in self._idx_to_token:
            raise KeyError("Vocabulary에 인덱스(%d)가 없습니다." % index)
        return self._idx_to_token[index]

# =======================================

    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)

    def __len__(self):
        return len(self._token_to_idx)

### `ReviewVectorizer` 클래스

텍스트를 수치 벡터로 변환하고, 어휘 사전을 생성하고 관리하는 클래스입니다.

입력 데이터의 토큰을 순회하며 각 토큰을 정수(원-핫 인코딩된 벡터)로 바꿔줍니다. 항상 길이가 일정한 벡터를 출력하며, 위에서 만든 `Vocabulary` 클래스
 중 토큰-인덱스 매핑인 `lookup_token`을 참고합니다.

데이터셋을 변형하여 순환 신경망에 주입하는 방법에는 원-핫 인코딩과 단어 임베딩이 있었는데요, **원-핫 인코딩**을 사용하겠습니다.

`from_dataframe()`은 판다스 데이터 프레임을 순회하며
1. 데이터셋에 있는 모든 토큰의 개별 빈도수를 카운트
2. `cutoff`에서 지정한 25회보다 빈도가 높은 토큰를 모두 찾아 `Vocabulary` 객체에 추가합니다.

In [ ]:
class ReviewVectorizer(object):
    """ 어휘 사전을 생성하고 관리합니다 """
    def __init__(self, review_vocab, rating_vocab):
        """
        매개변수:
            review_vocab (Vocabulary): 단어를 정수에 매핑하는 Vocabulary
            rating_vocab (Vocabulary): 클래스 레이블을 정수에 매핑하는 Vocabulary
        """
        self.review_vocab = review_vocab
        self.rating_vocab = rating_vocab

# =======================================
# Q9 모두 self.review_vocab 길이만큼 0으로 채운 벡터를 만듭니다.
# Q10 각각의 단어에 해당하는 위치가 1이 되어야겠죠? 0을 1로 대체합니다.
# Q11 무엇을 반환해야 할까요?
# =======================================

    def vectorize(self, review):
        """ 리뷰에 대한 웟-핫 벡터를 만듭니다
        
        매개변수:
            review (str): 리뷰
        반환값:
            one_hot (np.ndarray): 원-핫 벡터
        """
        one_hot = np.zeros(len(self.review_vocab), dtype=np.float32)
        
        for token in review.split(" "):
            if token not in string.punctuation:
                one_hot[self.review_vocab.lookup_token(token)] = 1

        return one_hot

    @classmethod
    def from_dataframe(cls, review_df, cutoff=25):
        """ 데이터셋 데이터프레임에서 Vectorizer 객체를 만듭니다
        
        매개변수:
            review_df (pandas.DataFrame): 리뷰 데이터셋
            cutoff (int): 빈도 기반 필터링 설정값
        반환값:
            ReviewVectorizer 객체
        """
        review_vocab = Vocabulary(add_unk=True)
        rating_vocab = Vocabulary(add_unk=False)
        
        # 점수를 추가합니다
        for rating in sorted(set(review_df.rating)):
            rating_vocab.add_token(rating)

        # count > cutoff인 단어를 추가합니다
        word_counts = Counter()
        for review in review_df.review:
            for word in review.split(" "):
                if word not in string.punctuation:
                    word_counts[word] += 1
               
        for word, count in word_counts.items():
            if count > cutoff:
                review_vocab.add_token(word)

        return cls(review_vocab, rating_vocab)
        
    @classmethod
    def from_serializable(cls, contents):
        """ 직렬화된 딕셔너리에서 ReviewVectorizer 객체를 만듭니다
        
        매개변수:
            contents (dict): 직렬화된 딕셔너리
        반환값:
            ReviewVectorizer 클래스 객체
        """
        review_vocab = Vocabulary.from_serializable(contents['review_vocab'])
        rating_vocab =  Vocabulary.from_serializable(contents['rating_vocab'])

        return cls(review_vocab=review_vocab, rating_vocab=rating_vocab)

    def to_serializable(self):
        """ 캐싱을 위해 직렬화된 딕셔너리를 만듭니다
        
        반환값:
            contents (dict): 직렬화된 딕셔너리
        """
        return {'review_vocab': self.review_vocab.to_serializable(),
                'rating_vocab': self.rating_vocab.to_serializable()}

### `DataLoader` 클래스
앞에서 벡터로 변환된 데이터 포인트를 미니배치로 모아줍니다. (신경망 훈련 시 필수)

튜토리얼: https://tutorials.pytorch.kr/beginner/basics/data_tutorial.html?highlight=dataloader

In [ ]:
# =======================================
# Q12
# =======================================

def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"):
    """
    파이토치 DataLoader를 감싸고 있는 제너레이터 함수.
    걱 텐서를 지정된 장치로 이동합니다.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

## 3) `ReviewClassifier` 모델 만들기
지금까지 단어-인덱스를 매핑하는 사전을 만들고 사전을 활용하여 리뷰 문장을 벡터화했습니다. 이번에는 가공한 문장으로 모델을 학습시키겠습니다. 두 가지 메서드를 정의해줍니다.
- (1) __init()은 모듈이 어떤 하위 모듈을 사용하는지 정의하는 constructor 함수입니다. 여기서는 ReviewClassifier라는 상위 모듈이 rnn과 fc(linear)를 하위 모듈로 사용합니다. batch_first=True 이면 입력, 출력 텐서 모두 (batch, seq, feature) 형태입니다.
- (2) forward()는 x_in이라는 입력을 받은 네트워크의 계산방식을 정의합니다. (정방향 계산)


In [ ]:
# =======================================
# Q13, 14, 15, 16 input_dim, hidden_dim, output_dim 중 알맞은 차원을 대입해주세요.
# =======================================
class ReviewClassifier(nn.Module):

    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(ReviewClassifier, self).__init__()
        self.rnn = nn.RNN(input_dim, # 입력 데이터의 특성 차원을 받아서
                          hidden_dim, # hidden state 차원을 출력
                          num_layers, 
                          batch_first=True) 
        
        self.fc = nn.Linear(hidden_dim, # fc의 입력값은 self.rnn의 출력값과 동일함
                            output_dim, # 리뷰 데이터의 클래스 개수를 반환
                            bias=True)
        
    def forward(self, x_in):
        """        
        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서 
                x_in.shape는 (batch, num_features)입니다.
        반환값:
            결과 텐서. tensor.shape은 (batch,)입니다.
        """
        out, hidden = self.rnn(x_in)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out) # fully connected layer를 지남
        return out

뒷부분에서 시그모이드 함수가 포함된 형태인 `nn.BCEWithLogitsLoss`를 사용하기 때문에 모델 구축 시 시그모이드 함수를 생략했습니다. 만약 `nn.BCELoss`를 사용한다면 `__init__` 마지막에 `self.act = nn.Sigmoid()`를 추가하고 `forward return` 전에 `out = self.act(out)`를 추가해야 합니다.

## 4) 훈련 과정

### 헬퍼 함수

헬퍼 함수는 훈련을 도와줍니다.
- `make_train_state()`: 훈련 과정 중 훈련 상태를 저장할 변수들의 묶음을 만든다.
- `update_train_state()`: 훈련 상태를 업데이트한다. 성능이 향상되면 현재 모델을 저장하여 최상의 모델을 사용할 수 있도록 한다.
- `compute_accuracy()`: 정확도를 계산한다.

In [ ]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}

def update_train_state(args, model, train_state):
    """ 훈련 상태를 업데이트합니다.

    Components:
     - 조기 종료: 과대 적합 방지
     - 모델 체크포인트: 더 나은 모델을 저장합니다

    :param args: 메인 매개변수
    :param model: 훈련할 모델
    :param train_state: 훈련 상태를 담은 딕셔너리
    :returns:
        새로운 훈련 상태
    """

# =======================================
# Q17 조기종료 코드입니다. 부등호 방향이 어떻게 될까요? (<, >)
# =======================================

    # 적어도 한 번 모델을 저장합니다
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # 성능이 향상되면 모델을 저장합니다
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]

        # 손실이 나빠지면
        if loss_t >= train_state['early_stopping_best_val']:
            # 조기 종료 단계 업데이트
            train_state['early_stopping_step'] += 1
        # 손실이 감소하면
        else:
            # 최상의 모델 저장
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])

            # 조기 종료 단계 재설정
            train_state['early_stopping_step'] = 0

        # 조기 종료 여부 확인
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

def compute_accuracy(y_pred, y_target):
    y_target = y_target.cpu()
    y_pred_indices = (torch.sigmoid(y_pred)>0.5).cpu().long()#.max(dim=1)[1]
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

### 초기화

Vectorizer를 준비합니다.
1. 위에서 만든 분류 모델 `ReviewClassifier` 객체를 생성합니다.
2. loss function는 `BCEWithLogitsLoss`, optimizer는 `Adam`으로 설정합니다.
3. `lr_scheduler`로 학습 과정에서 learning rate를 조정할 수 있도록 합니다.

참고) `nn.BCEWithLogitsLoss(): nn.BCELoss() (Binary Cross Entropy Loss) 에 Sigmoid 함수가 포함된 형태`

In [ ]:
if args.reload_from_files:
    # 체크포인트에서 훈련을 다시 시작
    print("데이터셋과 Vectorizer를 로드합니다")
    dataset = ReviewDataset.load_dataset_and_load_vectorizer(args.review_csv,
                                                            args.vectorizer_file)
else:
    print("데이터셋을 로드하고 Vectorizer를 만듭니다")
    # 데이터셋과 Vectorizer 만들기
    dataset = ReviewDataset.load_dataset_and_make_vectorizer(args.review_csv)
    dataset.save_vectorizer(args.vectorizer_file) 

vectorizer = dataset.get_vectorizer()

classifier = ReviewClassifier(input_dim=len(vectorizer.review_vocab),
                              hidden_dim=100,
                              output_dim=1, # 클래스가 0, 1이므로 1-dim
                              num_layers=2, # RNN layer 2개를 stacking하여 입력 데이터가 layer 2개를 차례대로 지나도록 합니다.
                            )
classifier = classifier.to(args.device)

loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                 mode='min', factor=0.5,
                                                 patience=1)
train_state = make_train_state(args)

데이터셋을 로드하고 Vectorizer를 만듭니다


### 훈련 반복

튜토리얼: https://tutorials.pytorch.kr/beginner/basics/optimization_tutorial.html#optimizer



In [ ]:
# tqdm: 진행률 프로세스바 시각화
epoch_bar = tqdm.notebook.tqdm(desc='training routine', 
                          total=args.num_epochs,
                          position=0)

dataset.set_split('train')
train_bar = tqdm.notebook.tqdm(desc='split=train',
                          total=dataset.get_num_batches(args.batch_size), 
                          position=1, 
                          leave=True)
dataset.set_split('val')
val_bar = tqdm.notebook.tqdm(desc='split=val',
                        total=dataset.get_num_batches(args.batch_size), 
                        position=1, 
                        leave=True)

# 훈련 반복
# =======================================
# Q23, 24
# =======================================

try:
    for epoch_index in range(args.num_epochs):
        # 현재 에포크 인덱스 저장
        train_state['epoch_index'] = epoch_index

        # <1> 훈련 세트에 대한 순회
        # 훈련 세트와 배치 제너레이터 준비, 손실과 정확도를 0으로 설정
        dataset.set_split('train')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            # 훈련 과정은 5단계로 이루어집니다

            # --------------------------------------
            # 단계 1. 그레이디언트(기울기)를 0으로 초기화합니다
            optimizer.zero_grad()

            # 단계 2. 출력을 계산합니다
            y_pred = classifier(x_in=batch_dict['x_data'].float()).squeeze(1)

            # 단계 3. 2단계에서 계산한 출력과 true label(y_target)을 비교해 손실을 계산합니다
            loss = loss_func(y_pred, batch_dict['y_target'].float())
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # 단계 4. 손실을 사용해 그레이디언트를 계산합니다
            loss.backward()

            # 단계 5. 옵티마이저로 가중치를 업데이트합니다
            optimizer.step()
            # -----------------------------------------
            
            # 정확도를 계산합니다
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            # 진행 바 업데이트
            train_bar.set_postfix(loss=running_loss, 
                                  acc=running_acc, 
                                  epoch=epoch_index)
            train_bar.update()

        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)

        # <2> 검증 세트에 대한 순회

        # 검증 세트와 배치 제너레이터 준비, 손실과 정확도를 0으로 설정
        dataset.set_split('val')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.
        running_acc = 0.
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):

            # 단계 1. 출력을 계산합니다
            y_pred = classifier(x_in=batch_dict['x_data'].float()).squeeze(1)

            # 단계 2. 손실을 계산합니다
            loss = loss_func(y_pred, batch_dict['y_target'].float())
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # 단계 3. 정확도를 계산합니다
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)
            
            val_bar.set_postfix(loss=running_loss, 
                                acc=running_acc, 
                                epoch=epoch_index)
            val_bar.update()

        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)

        train_state = update_train_state(args=args, model=classifier,
                                         train_state=train_state)

        scheduler.step(train_state['val_loss'][-1])

        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()

        if train_state['stop_early']:
            break

        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()
except KeyboardInterrupt:
    print("Exiting loop")

training routine:   0%|          | 0/100 [00:00<?, ?it/s]

split=train:   0%|          | 0/306 [00:00<?, ?it/s]

split=val:   0%|          | 0/65 [00:00<?, ?it/s]

에포크 10은 약 2분 소요됩니다. 저는 성능을 높이고자 에포크 100을 설정하여 약 20분 소요되었습니다.

학습시킨 모델을 테스트 데이터로 평가합니다.

바로 위에서 만든 `update_train_state()` 함수를 통해 학습 과정에서 성능이 가장 좋은 모델을 저장했습니다. 이 모델은 train_state['model_filename']의 위치에 저장되어 있습니다. 이 파일을 불러와 가장 성능이 좋은 모델을 가져옵시다.

In [ ]:
# 가장 좋은 모델을 사용해 테스트 세트의 손실과 정확도를 계산합니다
classifier.load_state_dict(torch.load(train_state['model_filename']))
classifier = classifier.to(args.device)

dataset.set_split('test')
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # 출력을 계산합니다
    y_pred = classifier(x_in=batch_dict['x_data'].float()).squeeze(1)

    # 손실을 계산합니다
    loss = loss_func(y_pred, batch_dict['y_target'].float())
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # 정확도를 계산합니다
    acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

테스트 손실과 정확도는 각각 train_state의 test_loss와 test_acc에 저장되어 있습니다. 출력해봅시다.

In [ ]:
print("테스트 손실: {:.3f}".format(train_state['test_loss']))
print("테스트 정확도: {:.2f}".format(train_state['test_acc']))

**Q27. 결과를 간단하게 적어주세요.**

- (답)

정확도가 너무 낮아 실망하실 수 있겠지만, 에포크 수를 100으로 늘린다면 정확도를 약 90%까지 향상시킬 수 있습니다. 출력된 결과는 모두 에포크 100으로 시도한 것입니다.

### 추론

학습된 모델에 새로운 리뷰 텍스트를 입력하여 긍정/부정을 판단해봅시다.

In [ ]:
# 텍스트 전처리 함수
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text

In [ ]:
def predict_rating(review, classifier, vectorizer, decision_threshold=0.5):
    """ 리뷰 점수 예측하기
    
    매개변수:
        review (str): 리뷰 텍스트
        classifier (ReviewClassifier): 훈련된 모델
        vectorizer (ReviewVectorizer): Vectorizer 객체
        decision_threshold (float): 클래스를 나눌 결정 경계
    """
    review = preprocess_text(review)
    
    vectorized_review = torch.tensor(vectorizer.vectorize(review))
    result = classifier(vectorized_review.view(1, -1))
    
    probability_value = torch.sigmoid(result).item()
    index = 1
    if probability_value < decision_threshold:
        index = 0

    return vectorizer.rating_vocab.lookup_index(index)

원하는 문장을 작성해 결과를 확인해보세요!

(에포크 10은 정확도가 50%에 불과하기 때문에 모델이 기대만큼 똑똑하지 않을 수 있습니다. 보다 정확한 분류를 원한다면 에포크 수를 늘려서 시도해보세요.)

In [ ]:
test_review = "this is a pretty awesome food"

classifier = classifier.cpu()
prediction = predict_rating(test_review, classifier, vectorizer, decision_threshold=0.5)
print("{} -> {}".format(test_review, prediction))

this is a pretty awesome food -> positive


### 해석
마지막으로 긍정 리뷰에 영향을 미치는 단어와 부정 리뷰에 영향을 미치는 단어를 확인해보겠습니다.

In [ ]:
classifier.fc.weight.shape

torch.Size([1, 100])

In [ ]:
# 가중치 정렬
fc_weights = classifier.fc.weight.detach()[0]
_, indices = torch.sort(fc_weights, dim=0, descending=True)
indices = indices.numpy().tolist()

# 긍정적인 상위 20개 단어
print("긍정 리뷰에 영향을 미치는 단어:")
print("--------------------------------------")
for i in range(20):
    print(vectorizer.review_vocab.lookup_index(indices[i]))
    
print("====\n\n\n")

# 부정적인 상위 20개 단어
print("부정 리뷰에 영향을 미치는 단어:")
print("--------------------------------------")
indices.reverse()
for i in range(20):
    print(vectorizer.review_vocab.lookup_index(indices[i]))

긍정 리뷰에 영향을 미치는 단어:
--------------------------------------
left
which
find
stellar
next
waste
the
day
but
give
just
any
don
unless
fine
minutes
terrible
finished
an
went
====



부정 리뷰에 영향을 미치는 단어:
--------------------------------------
for
working
does
not
waited
delivered
roadhouse
business
my
simple
hadn
she
believe
rest
our
into
texas
could
review
upset


# 수고하셨습니다! 😃

참고한 자료)
- 책 '파이토치로 배우는 자연어 처리(한빛미디어) 소스코드: [전처리](https://github.com/rickiepark/nlp-with-pytorch/blob/main/chapter_3/3_5_yelp_dataset_preprocessing_LITE.ipynb), [감성 분류](https://github.com/rickiepark/nlp-with-pytorch/blob/main/chapter_3/3_5_Classifying_Yelp_Review_Sentiment.ipynb)
- 책 코드 리뷰 [1](
https://cocosy.tistory.com/62), [2](https://didu-story.tistory.com/87), [3](https://hajunyoo.oopy.io/922c7c5d-bbaa-4491-be89-638deca3329e)